# Preprocesado de Datos
 ## Se importan datos usando las funcionalidades de Numpy y Pandas
 
 ### Trabajo de Fin de Grado - Universidad de Burgos
 #### Jesús García Armario

In [1]:
## Imports
import pandas as pd
import numpy as np

In [68]:
# Se importa el fichero excel de intervenciones realizadas
df = pd.read_excel('rptListadoIntervRealizadas.xls',header = [9,10], skipfooter = 12,index_col=[0,1,2,3])
df = df.droplevel(1,axis=1)

# Se renombran las columnas repetidas
cols = []
countNH = 1
countNUHSA = 1
countNombre = 1
countTipo = 1
countTurno = 1
for i in df.columns:
    if i == 'NHC':
        cols.append('NHC_'+str(countNH))
        countNH += 1
        continue
    if i=='NUHSA':
        cols.append('NUHSA'+str(countNUHSA))
        countNUHSA += 1
        continue
    if i=='APELLIDOS, NOMBRE':
        cols.append('Nombre'+str(countNombre))
        countNombre += 1
        continue
    if i=='TIPO':
        cols.append('Tipo'+str(countTipo))
        countTipo += 1
        continue
    if i=='TURNO':
        cols.append('Turno'+str(countTurno))
        countTurno += 1
        continue
    cols.append(i)
df.columns = cols

# Se eliminan las columnas que no se van a utilizar
df = df.drop(['NHC_2','NHC_3','NHC_4','NHC_5','NUHSA2','NUHSA3','NUHSA4','Nombre2','Nombre3',
'Nombre4','Nombre5','Nombre6','Nombre7','Turno1','Tipo1','Tipo2','Tipo3'],axis=1)

# Se muestra por pantalla
df.head(10)

FECHA    NHC_1        NUHSA1  \
NaN HG P-3 QUIROFANO- NaN NaN                  NaN  3170501  AN0503355532   
                          NaN                  NaN      NaN          86.4   
                          NaN  2020-05-04 08:00:00      NaN           NaN   
                          NaN                  NaN  2484412  AN0456806040   
                          NaN                  NaN      NaN          86.4   
                          NaN  2019-07-04 16:05:00      NaN           NaN   
    HG P-4 QUIROFANO- NaN NaN                  NaN  3321891  AN1191300557   
                          NaN                  NaN      NaN           NaN   
                          NaN  2019-09-16 16:10:00      NaN           NaN   
                          NaN                  NaN  1918818  AN0589983606   

                                                          Nombre1 Turno2  \
NaN HG P-3 QUIROFANO- NaN NaN           GARCIA DELGADO, ESPERANZA    NaN   
                          NaN           ESCISION PIEL RADICAL (&)    NaN   
                          NaN                                 NaN    NaN   
                          NaN  RODRIGUEZ ESCRIBANO, JUAN LAUREANO    NaN   
                          NaN           ESCISION PIEL RADICAL (&)    NaN   
                          NaN                                 NaN    NaN   
    HG P-4 QUIROFANO- NaN NaN            RUBIO ORTIZ, JUAN MANUEL    NaN   
                          NaN              REDUCCION INSTRUMENTAL    NaN   
                          NaN                                 NaN    NaN   
                          NaN             GOMEZ CASTILLO, ENRIQUE    NaN   

                               Turno3  Turno4  Turno5   CARÁCTER ECONÓMICO  \
NaN HG P-3 QUIROFANO- NaN NaN     NaN     NaN     NaN  Actividad Ordinaria   
                          NaN     NaN     NaN     NaN                   94   
                          NaN     NaN     NaN     NaN                  NaN   
                          NaN     NaN     NaN     NaN  Actividad Ordinaria   
                          NaN     NaN     NaN     NaN                   39   
                          NaN     NaN     NaN     NaN                  NaN   
    HG P-4 QUIROFANO- NaN NaN     NaN     NaN     NaN  Actividad Ordinaria   
                          NaN     NaN     NaN     NaN                    3   
                          NaN     NaN     NaN     NaN                  NaN   
                          NaN     NaN     NaN     NaN  Actividad Ordinaria   

                               CARÁCTER ECONÓMICO  CARÁCTER ECONÓMICO  \
NaN HG P-3 QUIROFANO- NaN NaN                 NaN                 NaN   
                          NaN                 NaN                 NaN   
                          NaN                 NaN                 NaN   
                          NaN                 NaN                 NaN   
                          NaN                 NaN                 NaN   
                          NaN                 NaN                 NaN   
    HG P-4 QUIROFANO- NaN NaN                 NaN                 NaN   
                          NaN                 NaN                 NaN   
                          NaN                 NaN                 NaN   
                          NaN                 NaN                 NaN   

                               CARÁCTER ECONÓMICO  CARÁCTER ECONÓMICO INICIO  \
NaN HG P-3 QUIROFANO- NaN NaN                 NaN                 NaN    NaN   
                          NaN                  94                 NaN    NaN   
                          NaN                 NaN                 NaN    NaN   
                          NaN                 NaN                 NaN    NaN   
                          NaN                  39                 NaN    NaN   
                          NaN                 NaN                 NaN    NaN   
    HG P-4 QUIROFANO- NaN NaN                 NaN                 NaN    NaN   
                          NaN                   0        